# 연구 및 기사 작성을 위한 에이전트 생성하기

멀티 에이전트 시스템(multi-agent systems)의 기본 개념을 소개하고, CrewAI 프레임워크에 대한 개요를 제공합니다.

In [1]:
# pip install crewai crewai-tools
# %pip install 'crewai[tools]'

In [6]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [7]:
from crewai import Agent, Task, Crew

In [8]:
import os
from crew_ai_utils import pretty_print_result, get_openai_api_key

import dotenv

dotenv.load_dotenv()
openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-4o'
os.environ["SERPER_API_KEY"] = os.getenv("SERPER_API_KEY")


In [9]:
from crewai_tools import SerperDevTool


# Loading Tools
search_tool = SerperDevTool()

## 에이전트 생성
- 에이전트를 정의하고, `role` (역할), `goal` (목표) 및 `backstory` (배경 이야기) 정의
- LLM(Large Language Model)은 역할극을 할 때 성능이 더 좋은 것으로 알려져 있음

### Agent: Planner(콘텐츠 기획자)


In [10]:
planner = Agent(
    role="콘텐츠 기획자 (Content Planner)",
    goal="매력적이고 사실에 근거한 콘텐츠를 {topic}(주제)에 대해 기획합니다.",
    backstory="당신은 {topic}(주제)에 대한 블로그 기사 기획 작업을 하고 있습니다."
              "당신은 독자들에게 유용한 정보를 수집하여 독자들이 학습하고 "
              "정보에 입각한 결정을 내릴 수 있도록 돕습니다. "
              "당신의 작업은 콘텐츠 작가(Content Writer)가 이 주제에 대한 기사를 작성하는 데 기초가 됩니다.",
    allow_delegation=False,
    tools=[search_tool],
	verbose=True
)

### Agent: Writer(콘텐츠 작가)

In [11]:
writer = Agent(
    role="콘텐츠 작가 (Content Writer)",
    goal="주제({topic})에 대해 통찰력 있고 사실에 근거한 의견(opinion)을 작성합니다.",
    backstory="당신은 주제({topic})에 대한 새로운 의견(opinion)을 작성하고 있습니다. "
              "당신은 콘텐츠 기획자(Content Planner)의 작업, 즉 개요(outline)와 주제에 대한 관련 맥락(context)을 기반으로 글을 씁니다. "
              "당신은 콘텐츠 기획자(Content Planner)가 제공한 개요(outline)의 주요 목표와 방향을 따릅니다. "
              "또한 객관적이고 공정한 통찰력(insights)을 제공하고 콘텐츠 기획자(Content Planner)가 제공한 정보를 통해 이를 뒷받침합니다. "
              "당신은 당신의 진술이 객관적인 진술이 아닌 의견(opinions)일 때 의견(opinion)에서 인정합니다.",
    allow_delegation=False,
    verbose=True
)

### Agent: Editor(편집자)

In [12]:
editor = Agent(
    role="편집자 (Editor)",
    goal="제공된 블로그 게시물을 편집하여 조직의 글쓰기 스타일에 맞춥니다.",
    backstory="당신은 콘텐츠 작가(Content Writer)로부터 블로그 게시물을 받는 편집자입니다. "
              "당신의 목표는 블로그 게시물을 검토하여 저널리즘의 모범 사례를 따르고,"
              "의견이나 주장을 제시할 때 균형 잡힌 관점을 제공하고,"
              "가능한 경우 주요 논쟁 거리가 되는 주제나 의견을 피하는 것입니다.",
    allow_delegation=True,
    verbose=True
)

### Agent: Translator(번역자)

In [13]:
translator = Agent(
    role="번역자 (Translator)",
    goal="편집이 완료된 블로그 게시물을 요청한 언어로 정확하게 번역하여 사용자에게 제공합니다.",
    backstory="당신은 편집자(Editor)로부터 블로그 게시물을 받는 번역자입니다. "
              "당신은 다양한 언어에 능통한 숙련된 번역가이며, 특히 IT 도메인에 전문성이 있습니다. "
              "당신의 목표는 콘텐츠의 의미, 어조, 스타일을 유지하면서 텍스트를 정확하게 번역하는 것입니다. "
              "당신은 다양한 분야의 텍스트를 번역한 경험이 있으며, 각 언어의 문화적 뉘앙스를 이해하고 있습니다.",
    allow_delegation=False,
    verbose=True
)

## 작업 생성

- `task`, `description`, `expected_output`, `agent` 정의

### Task: Plan(콘텐츠 계획)

In [14]:
plan = Task(
    description=(
        "1. {topic}에 대한 최신 트렌드(latest trends), 주요 업체(key players) 및 주목할 만한 뉴스(noteworthy news)를 우선적으로 고려합니다.\n"
        "2. 대상 독자(target audience)의 관심사(interests)와 고충(pain points)을 고려하여 파악합니다.\n"
        "3. 서론(introduction), 핵심 내용(key points), 행동 촉구(call to action)를 포함한 상세한 콘텐츠 개요(content outline)를 작성합니다.\n"
        "4. SEO 키워드(SEO keywords)와 관련된 데이터(data) 또는 소스(sources)를 포함합니다.\n"
    ),
    expected_output="개요(outline), 대상 독자 분석(audience analysis), "
        "SEO 키워드(SEO keywords) 및 리소스(resources)를 포함하는 포괄적인 콘텐츠 계획 문서(comprehensive content plan document)입니다.",
    agent=planner,
    human_input=True
)

### Task: Write(콘텐츠 작성)

In [15]:
write = Task(
    description=(
        "1. 콘텐츠 계획(content plan)을 활용하여 {topic}에 대한 설득력 있는 블로그 게시물(blog post)을 작성합니다.\n"
        "2. SEO 키워드(SEO keywords)를 자연스럽게 통합합니다.\n"
        "3. 섹션/소제목(Sections/Subtitles)이 매력적인 방식으로 적절하게 명명되었는지 확인합니다.\n"
        "4. 게시물이 매력적인 서론(introduction), 통찰력 있는 본문(insightful body) 및 요약 결론(summarizing conclusion)으로 구성되었는지 확인합니다.\n"
        "5. 문법적 오류(grammatical errors)가 없는지, 브랜드의 목소리(brand's voice)와 일치하는지 교정합니다(Proofread).\n"
    ),
    expected_output="잘 작성된 블로그 게시물(blog post)을 마크다운 형식(markdown format)으로 제공하며, 각 섹션은 2~3개의 단락(paragraphs)으로 구성되어 출판 준비가 완료되어야 합니다.",
    agent=writer,
)

### Task: Edit(콘텐츠 편집)

In [16]:
edit = Task(
    description=(
        "1. 주어진 블로그 게시물(blog post)에서 문법적 오류(grammatical errors)를 교정하고 브랜드의 목소리(brand's voice)와 일치하는지 확인합니다.\n"
    ),
    expected_output=(
        "잘 작성된 블로그 게시물(blog post)을 마크다운 형식(markdown format)으로 제공하며, "
        "각 섹션은 2~3개의 단락(paragraphs)으로 구성되어 출판 준비가 완료되어야 합니다."
    ),
    agent=editor,
)

### Task: Translate(번역)

In [17]:
translate = Task(
    description=(
        "문법적 오류(grammatical errors)를 교정하고 브랜드의 목소리(brand's voice)와 일치하는지 확인이 끝나면 일본어로 번역합니다.\n"
    ),
    expected_output=(
        "일본어로 번역된 블로그 게시물(blog post)을 마크다운 형식(markdown format)으로 제공합니다."
    ),
    agent=translator,
)

## Crew 구성

- 에이전트(Agents)로 Crew를 구성
- 해당 에이전트(agents)가 수행할 작업(tasks)을 전달
    - 이 간단한 예제에서는* 작업(tasks)이 순차적으로 수행 됨.(즉, 서로 종속적)  
    - 따라서 목록에서 작업(task)의 _순서(order)_ 가 중요함.
- `verbose=True`를 설정하면 실행에 대한 모든 로그를 볼 수 있음

In [18]:
from crewai import Crew, Process
from langchain_openai import ChatOpenAI

crew = Crew(
    agents=[planner, writer, editor, translator],
    tasks=[plan, translate, write, edit],
    manager_llm=ChatOpenAI(model="gpt-4o", 
                           temperature=0.7),   
    process=Process.hierarchical,
    planning=True,
    planning_llm="gpt-4o",
    verbose=True
)

## Running the Crew

In [19]:
result = crew.kickoff(inputs={"topic": "인공지능"})

 
[2025-03-04 13:03:46][INFO]: Planning the crew execution
# Agent: Crew Manager
## Task: 1. 인공지능에 대한 최신 트렌드(latest trends), 주요 업체(key players) 및 주목할 만한 뉴스(noteworthy news)를 우선적으로 고려합니다.
2. 대상 독자(target audience)의 관심사(interests)와 고충(pain points)을 고려하여 파악합니다.
3. 서론(introduction), 핵심 내용(key points), 행동 촉구(call to action)를 포함한 상세한 콘텐츠 개요(content outline)를 작성합니다.
4. SEO 키워드(SEO keywords)와 관련된 데이터(data) 또는 소스(sources)를 포함합니다.
1. **Research Phase**: Use the 'Search the internet with Serper' tool to compile data on the latest trends in AI, key players, and noteworthy news. Make sure to cover diverse sources to get a comprehensive view. Keep these insights organized and updated for integration into the content plan. 

2. **Audience Analysis**: Identify and analyze target audience details such as their professional backgrounds, locations, and main interests, as well as any pain points they may encounter in relation to AI. Use findings to tailor content topics and tones.

3. **Content Outline De

- Display the results of your execution as markdown in the notebook.

In [20]:
from IPython.display import Markdown
Markdown(result.raw)

Title: Exploring AI Trends in 2023

**Introduction**

As artificial intelligence (AI) continues to evolve rapidly, 2023 marks another year of groundbreaking developments and intriguing challenges. This post delves into some of the most significant trends in AI, including advancements in Generative AI, strides in Natural Language Processing (NLP), the growing importance of ethical AI practices, and the innovative Crew AI framework. Each section offers an insight into how these trends are shaping our present and influencing the future.

**Generative AI: A Creative Leap**

Generative AI has emerged as a transformative force, enabling machines to generate content, from art to text, that mimics human creation. In 2023, we are witnessing a surge in sophisticated generative models, leveraging deep learning techniques to produce remarkably realistic outputs. These advancements are not only enhancing creative industries but also opening new avenues in virtual reality and immersive experiences.

Furthermore, as generative AI tools become more accessible, they are democratizing creativity, allowing individuals and small businesses to harness technology for innovation. Nevertheless, the evolution of generative AI also raises concerns about authenticity and copyright, prompting discussions on the need for robust regulatory frameworks.

**Natural Language Processing: Breaking New Ground**

Natural Language Processing (NLP) has seen transformative progress in 2023, with AI systems achieving unprecedented levels of comprehension and nuanced understanding of human language. Innovations in NLP are enabling applications such as real-time translation services and conversational agents that interact with users in a more human-like manner.

These advancements have significant implications for communication, offering businesses and individuals enhanced tools to bridge language barriers and foster global connectivity. However, as NLP technologies advance, addressing challenges around biases in language models and ensuring inclusivity and fairness remains critical.

**Ethical AI Practices: Towards Responsible Innovation**

As AI technologies become increasingly integrated into daily life, ethical AI practices are of paramount importance in 2023. Stakeholders across the spectrum are emphasizing the need to develop and deploy AI systems that prioritize privacy, transparency, and accountability. This commitment to ethical standards is vital to building trust and ensuring the long-term viability of AI applications.

Organizations are adopting frameworks to assess and mitigate potential biases and unintended consequences associated with AI systems. By fostering a culture of responsible innovation, the tech industry aims to align the capabilities of AI with overarching societal values and goals.

**Crew AI Framework: The Future of Collaboration**

The Crew AI framework represents a groundbreaking approach to collaborative intelligence, enabling teams to work seamlessly alongside AI systems. Introduced as a paradigm shift in 2023, Crew AI focuses on enhancing human-machine collaboration, optimizing workflows, and augmenting human decision-making processes.

This framework is particularly impactful in environments requiring rapid, informed decisions, such as healthcare and emergency response scenarios. By leveraging the strengths of both humans and machines, Crew AI promises to elevate productivity and achieve outcomes that neither could reach independently.

**Conclusion**

The AI trends of 2023 highlight an exhilarating journey into the future, marked by significant advancements and new challenges. As generative AI expands creative horizons, NLP breaks down language barriers, ethical practices guide responsible innovation, and Crew AI redefines collaboration, the potential benefits are immense. Embracing these developments with a focus on responsibility and inclusivity will ensure that AI continues to be a force for good, enriching our lives across various spheres.

This blog post has been carefully revised for grammatical accuracy and alignment with our brand's voice, ensuring it is polished and ready for publication. Each section is structured to provide a clear, insightful exploration of its respective topic.

## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [15]:
topic = "2025년 3월 2일 서울 날씨"
result = crew.kickoff(inputs={"topic": topic})

# Agent: 콘텐츠 작가 (Content Writer)
## Task: 1. 콘텐츠 계획(content plan)을 활용하여 2025년 3월 2일 서울 날씨에 대한 설득력 있는 블로그 게시물(blog post)을 작성합니다.
2. SEO 키워드(SEO keywords)를 자연스럽게 통합합니다.
3. 섹션/소제목(Sections/Subtitles)이 매력적인 방식으로 적절하게 명명되었는지 확인합니다.
4. 게시물이 매력적인 서론(introduction), 통찰력 있는 본문(insightful body) 및 요약 결론(summarizing conclusion)으로 구성되었는지 확인합니다.
5. 문법적 오류(grammatical errors)가 없는지, 브랜드의 목소리(brand's voice)와 일치하는지 교정합니다(Proofread).

# Agent: 콘텐츠 기획자 (Content Planner)
## Task: What are the SEO keywords we need to integrate naturally into the blog post about 2025년 3월 2일 서울 날씨, and can you provide more details on the brand's voice for consistency?


# Agent: 콘텐츠 기획자 (Content Planner)
## Final Answer: 
To effectively plan for the blog post about the weather in Seoul on March 2, 2025, incorporating relevant SEO keywords is crucial for maximizing search engine visibility. Here are some suggested keywords to integrate naturally into the content:

1. **2025 Seoul Weather**
2. **March 2 Weather Forecast Seoul**


In [17]:
from IPython.display import Markdown
Markdown(result.raw)

The comprehensive content plan for the 2025년 3월 2일 서울 날씨 blog post includes a detailed outline (introduction, key points, conclusion, call to action), an audience analysis outlining interests and pain points, a list of SEO keywords, and suggested resources for information and inspiration.